In [ ]:
pip install cdqa

In [1]:
from cdqa.pipeline import QAPipeline

/home/mshar004/.local/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/mshar004/.local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from cdqa.pipeline import QAPipeline

In [3]:
from cdqa.utils.download import download_model

In [41]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [42]:
data = pd.read_csv('newClassificationData.csv')

In [43]:
X = data['title']
y = data['label']

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [45]:
# Build the model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model using the training data
model.fit(X_train, y_train)

# Predict the categories of the test data
predicted_categories = model.predict(X_test)

/home/mshar004/.local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1278: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [46]:
print("The accuracy is {}".format(accuracy_score(y_test, predicted_categories)))

The accuracy is 0.8932330827067669


In [47]:
import matplotlib.pyplot as plt
from sklearn import metrics

precisions_lr, recalls_lr, thresholds = metrics.precision_recall_curve(y_test,
                                                                 model.predict_proba(X_test).T[1])

In [48]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_categories))

              precision    recall  f1-score   support

           0       0.95      0.85      0.89       353
           1       0.85      0.95      0.89       312

    accuracy                           0.89       665
   macro avg       0.90      0.90      0.89       665
weighted avg       0.90      0.89      0.89       665



In [367]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 489.6 MB 13 kB/s s eta 0:00:01  |▏                               | 1.8 MB 2.0 MB/s eta 0:04:09     |▏                               | 3.3 MB 2.0 MB/s eta 0:04:08     |▎                               | 3.7 MB 2.0 MB/s eta 0:04:08     |▊                               | 11.8 MB 10.4 MB/s eta 0:00:46     |█▌                              | 23.4 MB 14.8 MB/s eta 0:00:32     |██████▎                         | 96.3 MB 23.9 MB/s eta 0:00:17     |██████▍                         | 97.9 MB 23.9 MB/s eta 0:00:17     |██████▋                         | 100.9 MB 23.9 MB/s eta 0:00:17     |██████▊                         | 103.0 MB 13.5 MB/s eta 0:00:29     |██████▊                         | 103.4 MB 13.5 MB/s eta 0:00:29     |██████▉                         | 104.3 MB 13.5 MB/s eta 0:00:29     |███████▌                        | 114.0 MB 13.5 MB/s eta 0:00:28     |████████▊             

In [371]:
pip install tensorflow_text

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.9 MB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

In [17]:
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [18]:
l1 = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l2 = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l1)

In [19]:
model = tf.keras.Model(inputs=[text_input], outputs = [l2])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [21]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [22]:
model.fit(X_train, y_train, epochs=18)

Epoch 1/18
43/43 [==============================] - 529s 12s/step - loss: 0.6552 - accuracy: 0.6113 - precision: 0.6095 - recall: 0.6696
Epoch 2/18
43/43 [==============================] - 529s 12s/step - loss: 0.5638 - accuracy: 0.7381 - precision: 0.7286 - recall: 0.7783
Epoch 3/18
43/43 [==============================] - 537s 12s/step - loss: 0.5220 - accuracy: 0.7760 - precision: 0.7526 - recall: 0.8377
Epoch 4/18
43/43 [==============================] - 527s 12s/step - loss: 0.4819 - accuracy: 0.8079 - precision: 0.7839 - recall: 0.8623
Epoch 5/18
43/43 [==============================] - 514s 12s/step - loss: 0.4551 - accuracy: 0.8160 - precision: 0.8044 - recall: 0.8464
Epoch 6/18
43/43 [==============================] - 506s 12s/step - loss: 0.4351 - accuracy: 0.8368 - precision: 0.8282 - recall: 0.8594
Epoch 7/18
43/43 [==============================] - 500s 12s/step - loss: 0.4186 - accuracy: 0.8420 - precision: 0.8236 - recall: 0.8797
Epoch 8/18
43/43 [=======================

In [23]:
model.evaluate(X_test, y_test)

21/21 [==============================] - 257s 12s/step - loss: 0.3408 - accuracy: 0.8647 - precision: 0.8208 - recall: 0.9103


[0.34082257747650146,
 0.8646616339683533,
 0.8208092451095581,
 0.9102563858032227]

In [24]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [25]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,

In [26]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

array([[291,  62],
       [ 28, 284]])

In [27]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.91      0.82      0.87       353
           1       0.82      0.91      0.86       312

    accuracy                           0.86       665
   macro avg       0.87      0.87      0.86       665
weighted avg       0.87      0.86      0.86       665



In [11]:
pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [49]:
import requests
from bs4 import BeautifulSoup

In [99]:
url = "https://news.yahoo.com/"
page =  requests.get(url)  # Use requests.get to download the page.

In [100]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
headlines = soup.find_all('h3', attrs={'class': 'Mb(5px)'})

In [102]:
headlines

[<h3 class="Mb(5px)" data-reactid="62"><a class="js-content-viewer wafer-caas Fw(b) Fz(20px) Lh(23px) Fz(17px)--sm1024 Lh(19px)--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled" data-reactid="63" data-uuid="e80b88ae-26bd-331a-8b2b-e5015cbbf029" data-wf-caas-prefetch="1" data-wf-caas-uuid="e80b88ae-26bd-331a-8b2b-e5015cbbf029" href="/ring-doorbell-footage-captures-potentially-190739620.html"><u class="StretchedBox" data-reactid="64"></u><!-- react-text: 65 -->Ring doorbell footage captures potentially deadly new TikTok trend<!-- /react-text --></a></h3>,
 <h3 class="Mb(5px)" data-reactid="76"><a class="js-content-viewer wafer-caas Fw(b) Fz(20px) Lh(23px) Fz(17px)--sm1024 Lh(19px)--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled" data-reactid="77" data-uuid="25b78530-c1ad-3e0a-8216-dc3db5fcb1dd" data-wf-caas-prefetch="1" data-wf-caa

In [103]:
titles = [headline.text for headline in headlines]

In [104]:
titles

['Ring doorbell footage captures potentially deadly new TikTok trend',
 '‘What the hell is this?’: Golf course employee shares bizarre tip she got from customer',
 'Herschel Walker’s son protests high gas prices, but some say his $1,300 sweatshirt shows he’s full of hot air',
 "Woman exposes 'hidden killer' that's popular in old houses: 'Total death traps'",
 'These Teens Found A Brilliant Way To Prove School Dress Codes Are Sexist And Others Pranked An Entire School Board — Here Are 6 Other Things Youths Did To Win The Internet This Year',
 '‘We all went flying,’ Kentucky family says after tornado sucked them out of bathroom',
 "J.K. Rowling accused of perpetuating a 'false narrative' that trans women are 'predators' in latest tweet",
 'Pizzeria owner knocked unconscious while breaking up fight, North Carolina video shows',
 "Investigators at the scene after Russian teenager 'blows himself up' at Orthodox school",
 "Danish ex-minister convicted in 'child bride' impeachment case",
 'Pe

In [54]:
links = ["https://news.yahoo.com" + link.select('a')[0]['href'] for link in headlines]

In [55]:
links

['https://news.yahoo.com/ring-doorbell-footage-captures-potentially-190739620.html',
 'https://news.yahoo.com/hell-golf-course-employee-shares-181504802.html',
 'https://news.yahoo.com/m/2009b8e1-cdfc-3718-a4ba-461d5148c2f2/herschel-walker%E2%80%99s-son.html',
 'https://news.yahoo.com/homeowner-shares-she-walks-down-162545432.html',
 'https://news.yahoo.com/8-kids-teens-made-us-184602118.html',
 'https://news.yahoo.com/went-flying-kentucky-family-says-193137396.html',
 'https://news.yahoo.com/jk-rowling-accused-of-perpetuating-a-false-narrative-that-trans-women-are-predators-in-latest-tweet-200447279.html',
 'https://news.yahoo.com/pizzeria-owner-knocked-unconscious-while-172255681.html',
 'https://news.yahoo.com/investigators-scene-russian-teenager-blows-170126936.html',
 'https://news.yahoo.com/danish-ex-minister-convicted-child-142023820.html',
 'https://news.yahoo.com/pennsylvania-man-planted-explosives-blm-190100238.html',
 'https://news.yahoo.com/column-rand-paul-demands-federal-

In [56]:
df = pd.DataFrame(list(zip(titles, links)), columns =['titles', 'links'])

In [57]:
X = df['titles']

In [58]:
predicted_categories = model.predict(X)

In [59]:
df['predicted_label'] = predicted_categories

In [60]:
df2 = df[df['predicted_label'] == 1]

In [61]:
df2

,titles,links,predicted_label
0,Ring doorbell footage captures potentially dea...,https://news.yahoo.com/ring-doorbell-footage-c...,1
1,‘What the hell is this?’: Golf course employee...,https://news.yahoo.com/hell-golf-course-employ...,1
4,These Teens Found A Brilliant Way To Prove Sch...,https://news.yahoo.com/8-kids-teens-made-us-18...,1
16,Miss India Wins Miss Universe 2021,https://news.yahoo.com/miss-india-wins-miss-un...,1
17,"‘Don’t Say Anything Stupid, Because You’re Jus...",https://news.yahoo.com/don-t-anything-stupid-b...,1
18,Milwaukee car thieves are crazy for Hyundais a...,https://news.yahoo.comhttps://www.autoblog.com...,1
21,"Miss Universe is India's Harnaaz Sandhu, 70th ...",https://news.yahoo.com/harnaaz-sandhu-india-na...,1
22,18 Horrific Bosses In 2021 That Made People Sa...,https://news.yahoo.com/18-horrific-bosses-2021...,1
23,Here's What 30 Days of Mountain Climbers Did t...,https://news.yahoo.com/heres-30-days-mountain-...,1


In [62]:
import time
textl = []
for link in df2['links']:
    
    page = ''
    while  page == '':
        try:
            page = requests.get(link)
            if page != '':
                soup = BeautifulSoup(page.text, "html.parser")
                text = soup.get_text()
                sep = 'Recommended Stories'
                stripped = text.split(sep, 1)[0]
                textl.append(stripped)
            break
        except:
            #df2['link'] = pd.nan
            df2 = df2[df2['links']!= link]
            break   

In [63]:
textli = [[text] for text in textl]
textli[2]

['These Teens Found A Brilliant Way To Prove School Dress Codes Are Sexist And Others Pranked An Entire School Board — Here Are 6 Other Things Youths Did To Win The Internet This Year        HOME    MAIL    NEWS    FINANCE    SPORTS    ENTERTAINMENT    LIFE    SHOPPING    YAHOO PLUS    MORE...     Download the Yahoo News app          Yahoo News  Search query    Sign in   Mail Sign in to view your mail    News   News   US   US   Politics   Politics   World   World   COVID-19   COVID-19   Climate Change   Climate Change  Originals   Originals   The 360The 360 SkullduggerySkullduggery ConspiracylandConspiracyland     Health   Health   Science   Science   Podcasts   Podcasts   Contact Us   Contact Us   Videos   Videos   Autos   Autos  …… \n\nClose this contentBuzzFeedThese Teens Found A Brilliant Way To Prove School Dress Codes Are Sexist And Others Pranked An Entire School Board — Here Are 6 Other Things Youths Did To Win The Internet This YearRead full articleDecember 13, 2021, 1:46 PM·3

In [64]:
text_list = []
for i in range(len(textli)):
    ps = textli[i][0].split("\n")
    #print(ps)
    text_list.append([[p] for p in ps if [p]!=['']])
    #print(text_list)

text_list[2]

[['These Teens Found A Brilliant Way To Prove School Dress Codes Are Sexist And Others Pranked An Entire School Board — Here Are 6 Other Things Youths Did To Win The Internet This Year        HOME    MAIL    NEWS    FINANCE    SPORTS    ENTERTAINMENT    LIFE    SHOPPING    YAHOO PLUS    MORE...     Download the Yahoo News app          Yahoo News  Search query    Sign in   Mail Sign in to view your mail    News   News   US   US   Politics   Politics   World   World   COVID-19   COVID-19   Climate Change   Climate Change  Originals   Originals   The 360The 360 SkullduggerySkullduggery ConspiracylandConspiracyland     Health   Health   Science   Science   Podcasts   Podcasts   Contact Us   Contact Us   Videos   Videos   Autos   Autos  …… '],
 ['Close this contentBuzzFeedThese Teens Found A Brilliant Way To Prove School Dress Codes Are Sexist And Others Pranked An Entire School Board — Here Are 6 Other Things Youths Did To Win The Internet This YearRead full articleDecember 13, 2021, 1:46 

In [122]:
df2['body'] = text_list
df2['links'][1]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'https://news.yahoo.com/hell-golf-course-employee-shares-181504802.html'

In [66]:
df_converted = df2[['titles', 'body']]

In [67]:
df_converted

,titles,body
0,Ring doorbell footage captures potentially dea...,[[Ring doorbell footage captures potentially d...
1,‘What the hell is this?’: Golf course employee...,[[‘What the hell is this?’: Golf course employ...
4,These Teens Found A Brilliant Way To Prove Sch...,[[These Teens Found A Brilliant Way To Prove S...
16,Miss India Wins Miss Universe 2021,[[Miss India Wins Miss Universe 2021 HO...
17,"‘Don’t Say Anything Stupid, Because You’re Jus...","[[‘Don’t Say Anything Stupid, Because You’re J..."
21,"Miss Universe is India's Harnaaz Sandhu, 70th ...","[[Miss Universe is India's Harnaaz Sandhu, 70t..."
22,18 Horrific Bosses In 2021 That Made People Sa...,[[18 Horrific Bosses In 2021 That Made People ...
23,Here's What 30 Days of Mountain Climbers Did t...,[[Here's What 30 Days of Mountain Climbers Did...


In [68]:
df_converted = df_converted.rename(columns={"titles": "title", "body": "paragraphs"})

In [69]:
df_converted.reset_index(inplace = True)

In [70]:
df_converted['paragraphs']

0    [[Ring doorbell footage captures potentially d...
1    [[‘What the hell is this?’: Golf course employ...
2    [[These Teens Found A Brilliant Way To Prove S...
3    [[Miss India Wins Miss Universe 2021        HO...
4    [[‘Don’t Say Anything Stupid, Because You’re J...
5    [[Miss Universe is India's Harnaaz Sandhu, 70t...
6    [[18 Horrific Bosses In 2021 That Made People ...
7    [[Here's What 30 Days of Mountain Climbers Did...
Name: paragraphs, dtype: object

In [71]:
df_converted

import re

s_nums = df_converted['paragraphs'][0]

for i in range(len(df_converted)):
    s_nums = df_converted['paragraphs'][i]
    df_converted['paragraphs'][i] = [[sen] for sen in re.split('\.|!|\?', s_nums[1][0]) if len(sen) > 2]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [123]:
df_converted

,title,paragraphs
0,Ring doorbell footage captures potentially dea...,[[Close this contentRead full articleIn The Kn...
1,‘What the hell is this?’: Golf course employee...,[[Close this contentThe Independent‘What the h...
2,These Teens Found A Brilliant Way To Prove Sch...,[[Close this contentBuzzFeedThese Teens Found ...
3,Miss India Wins Miss Universe 2021,[[Close this contentPeopleMiss India Wins Miss...
4,"‘Don’t Say Anything Stupid, Because You’re Jus...",[[Close this contentAtlanta Black Star‘Don’t S...
5,"Miss Universe is India's Harnaaz Sandhu, 70th ...",[[Close this contentAssociated PressMiss Unive...
6,18 Horrific Bosses In 2021 That Made People Sa...,[[Close this contentBuzzFeed18 Horrific Bosses...
7,Here's What 30 Days of Mountain Climbers Did t...,[[Close this contentMen's HealthHere's What 30...


In [73]:
from cdqa.utils.download import download_squad, download_model, download_bnpp_data

directory = '/home/mshar004'

# Downloading data
download_squad(dir=directory)
download_bnpp_data(dir=directory)

# Downloading pre-trained BERT fine-tuned on SQuAD 1.1
download_model('bert-squad_1.1', dir=directory)

train-v1.1.json already downloaded
dev-v1.1.json already downloaded

train-v2.0.json already downloaded
dev-v2.0.json already downloaded

bnpp_newsroom-v1.1.csv already downloaded

bert_qa.joblib already downloaded


In [74]:
# Downloading pre-trained DistilBERT fine-tuned on SQuAD 1.1
download_model('distilbert-squad_1.1', dir=directory)


distilbert_qa.joblib already downloaded


In [76]:
df_converted.drop('index', axis=1, inplace= True)
df_converted

,title,paragraphs
0,Ring doorbell footage captures potentially dea...,[[Close this contentRead full articleIn The Kn...
1,‘What the hell is this?’: Golf course employee...,[[Close this contentThe Independent‘What the h...
2,These Teens Found A Brilliant Way To Prove Sch...,[[Close this contentBuzzFeedThese Teens Found ...
3,Miss India Wins Miss Universe 2021,[[Close this contentPeopleMiss India Wins Miss...
4,"‘Don’t Say Anything Stupid, Because You’re Jus...",[[Close this contentAtlanta Black Star‘Don’t S...
5,"Miss Universe is India's Harnaaz Sandhu, 70th ...",[[Close this contentAssociated PressMiss Unive...
6,18 Horrific Bosses In 2021 That Made People Sa...,[[Close this contentBuzzFeed18 Horrific Bosses...
7,Here's What 30 Days of Mountain Climbers Did t...,[[Close this contentMen's HealthHere's What 30...


In [77]:
df_converted.to_csv('df_bert.csv')

In [78]:
import pandas as pd
from ast import literal_eval
from cdqa.pipeline import QAPipeline

df = pd.read_csv('df_bert.csv', converters={'paragraphs': literal_eval})

In [79]:
#cdqa_pipeline = QAPipeline(reader='bert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline = QAPipeline(reader='distilbert_qa.joblib')

In [82]:
df.drop('Unnamed: 0', axis=1, inplace = True)

In [83]:
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [92]:
df['title'][1]

'‘What the hell is this?’: Golf course employee shares bizarre tip she got from customer'

In [93]:
df['paragraphs'][1]

[['Close this contentThe Independent‘What the hell is this'],
 ['’: Golf course employee shares bizarre tip she got from customerRead full articleChelsea RitschelDecember 13, 2021, 10:15 AM·4 min readGolf course employee reveals man tipped her with three ripped $100 bills (TikTok / @cassholland)A golf course employee who serves drinks to the golfers has revealed the bizarre tip she received from a man on the course in an effort to convince her to get dinner with him'],
 ['Cassie Holland, who works as a beverage cart driver in Las Vegas, Nevada, recalled the encounter in a TikTok uploaded in September, where she revealed that the man gave her three $100 bills that were ripped in half, and promised to give her the other halves if she showed up for dinner'],
 ['Holland, who goes by the username @cassholland on TikTok, began the video by explaining that she was sharing the story because she wanted to know if it had ever happened to anyone else'],
 ['The golf course employee then held up th

In [94]:
query = df['title'][1]

In [95]:
prediction = cdqa_pipeline.predict(query, 5)

In [96]:
prediction

[('TikTok',
  '‘What the hell is this?’: Golf course employee shares bizarre tip she got from customer',
  'In a follow-up video, Holland revealed that the man who’d given her the ripped money had shown up at the golf course again, with the golf course employee claiming that he gave her the rest of the money after admitting that he’d seen her TikTok video',
  7.390853153337114),
 ('What the hell is this',
  '‘What the hell is this?’: Golf course employee shares bizarre tip she got from customer',
  'The golf course employee then held up three $100 bills that had been ripped in half as she said: “What the hell is this',
  5.729512101124358),
 ('bizarre tip',
  '‘What the hell is this?’: Golf course employee shares bizarre tip she got from customer',
  '’: Golf course employee shares bizarre tip she got from customerRead full articleChelsea RitschelDecember 13, 2021, 10:15 AM·4 min readGolf course employee reveals man tipped her with three ripped $100 bills (TikTok / @cassholland)A golf 

In [97]:
prediction[0][0]

'TikTok'

In [98]:
for i in range(len(prediction)):
    if(prediction[i][2] not in query and query not in prediction[i][2] and 'Close this content' not in prediction[i][2]
      and 'Read full article' not in prediction[i][2] and 'This article' not in prediction[i][2] and 'More from In The Know' not in prediction[i][2]
      and 'min read' not in prediction[i][2] and 'More content below' not in prediction[i][2]
      ):
        print(prediction[i][2])

In a follow-up video, Holland revealed that the man who’d given her the ripped money had shown up at the golf course again, with the golf course employee claiming that he gave her the rest of the money after admitting that he’d seen her TikTok video
The golf course employee then held up three $100 bills that had been ripped in half as she said: “What the hell is this
’ That’s when I got super mad
